In [1]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def process_image(imageName, resultName, params="--edge-thresh 10 --peak-thresh 5"):
    """ Process an image and save results in a file. """
    
    if imageName[-3:] != 'pgm':
        im = Image.open(imageName).convert('L')
        im.save('tmp.pgm')
        imageName = 'tmp.pgm'
        
    cmmd = str("sift " + imageName + " --output " + resultName + " " + params)
    os.system(cmmd)
    print 'processed', imageName, 'to', resultName

In [3]:
def read_features_from_file(fileName):
    """ Read features and return matrix form. """
    f = np.loadtxt(fileName)
    
    # return feature locations (first 4 components) and descriptors (last 128 components)
    return f[:, :4], f[:, 4:]

In [4]:
def write_features_to_file(fileName, locs, desc):
    """ Save feature location and descriptors to file. """
    np.savetxt(fileName, np.hstack((locs, desc)))

In [5]:
def plot_features(im, locs, circle=False):
    def draw_circle(c, r):
        t = np.arange(0, 1.01, 0.01) * 2 * np.pi
        x = r * np.cos(t) + c[0]
        y = r * np.sin(t) + c[1]
        plt.plot(x,y,'b',linewidth=2)
    
    plt.imshow(im)
    if circle:
        for p in locs:
            draw_circle(p[:2], p[2])
    else:
        plot(locs[:,0], locs[:,1], 'bo')
    plt.axis('off')

In [6]:
def match(desc1, desc2):
    """ For each descriptor in the first image, select its
        match in the second image. 
        Inputs are descriptors of 2 images. """
    
    # normalize
    desc1 = np.array([d/np.linalg.norm(d) for d in desc1])
    desc2 = np.array([d/np.linalg.norm(d) for d in desc2])
    
    dist_ratio = 0.6
    desc1_size = desc1.shape
    
    matchScores = np.zeros((desc1_size[0], 1), 'int')
    desc2t = desc2.T
    
    for i in range(desc1_size[0]):
        dotprods = np.dot(desc1[i, :], desc2t)
        dotprods = 0.9999 * dotprods
        ind = np.argsort(np.arccos(dotprods))
        
        if np.arccos(dotprods)[ind[0]] < dist_ratio * np.arccos(dotprods)[ind[1]]:
            matchScores[i] = int(ind[0])
    
    return matchScores
    

In [7]:
def match_twosided(desc1, desc2):
    """ Two sided symmetric version of match(). """
    
    match_12 = match(desc1, desc2)
    match_21 = match(desc2, desc1)
    
    # nonzero()[0] returns the indices of nonzero entries 
    idx_12 = match_12.nonzero()[0]
    
    # remove matches that are asymmetric
    for n in idx_12:
        if match_21[int(match_12[n])] != n:
            match_12[n] = 0
            
    return match_12
    

In [8]:
def appendImages(im1, im2):
    """ Return a new image that appends two images side-by-side. """
    
    # fill with zeros for image with smaller row size
    r1 = im1.shape[0]
    r2 = im2.shape[0]
    if r1 < r2:
        im1 = np.concatenate((im1, np.zeros(r2-r1, im1.shape[1])), axis=0)  # concatenate vertically
    elif r1 > r2:
        im2 = np.concatenate((im2, np.zeros(r1-r2, im2.shape[1])), axis=0)
        
    return np.concatenate((im1, im2), axis=1)    # now concatenate side-by-side

In [9]:
def plot_matches(im1, im2, loc1, loc2, matchScores, show_below=True):
    """ Join matches in the plot. """
    
    im3 = appendImages(im1, im2)
    if show_below:
        im3 = np.vstack((im3, im3))
        
    plt.imshow(im3)
    
    # join matched points, need to add the column number of im1 to im2
    cols1 = im1.shape[1]
    for i, m in enumerate(matchScores):
        if m > 0:
            m = m[0]    # originally, m = [some number]
            plt.plot([loc1[i][0], loc2[m][0]+cols1], [loc1[i][1], loc2[m][1]], 'c')
            
    plt.axis('off')
    

In [10]:
# imName1 = 'liberty1.jpg'
# imName2 = 'liberty2.jpg'
# im1 = np.array(Image.open(imName1).convert('L'))
# im2 = np.array(Image.open(imName2).convert('L'))
# process_image(imName1, 'liberty1.sift')
# process_image(imName2, 'liberty2.sift')
# loc1, desc1 = read_features_from_file('liberty1.sift')
# loc2, desc2 = read_features_from_file('liberty2.sift')

processed tmp.pgm to liberty1.sift
processed tmp.pgm to liberty2.sift


In [11]:
# score_12 = match_twosided(desc1, desc2)

In [12]:
# # newIm = appendImages(im1, im2)
# plot_matches(im1, im2, loc1, loc2, score_12, show_below=False)
# plt.gray()
# plt.show()